In [41]:
import random
random.seed(119)
import pandas as pd

In [42]:
def read(fn):
    fh = open(fn, 'rt')
    data = []
    for line in fh:
        line = line.split('\t')
        label = line[0].strip('!')
        temp = line[1].strip('!').split('<|endoftext|>')
        question, response = temp[0], temp[1]
        data.append((label, question, response))
    return data

In [43]:
men_data = read('../8-28_annotations-and-classifiers/vanilla_sample_random_equal_t1.csv')
men_data = pd.DataFrame(men_data)
men_data.columns = ['gender', 'question', 'response']

In [53]:
def pronouns(x):
    keywords = [' he ', ' him ', ' she ', 'her', ' they ', ' them ', ' it ', 'is that', ' this ']
    return any(i in x.lower() for i in keywords)

def has_or(x):
    return 'or' in x.lower()

def rhetoric(x):
    keywords = ['shouldn\'t', 'wouldn\'t', 'couldn\'t']
    return any(x.lower().startswith(i) for i in keywords)

def too_short(x):
    return len(x.split(' ')) <= 5

In [45]:
men_data

,gender,question,response
0,M,do?,:p I got downvoted too
1,M,was?,is now dead in flames
2,W,Doug?,You got the wrong number!
3,W,Is it?,Of course. He was one of the first.
4,W,is it?,Is it an American accent?
...,...,...,...
59995,W,Did you seriously downvote me because I cried ...,"yes i did, just have to type"
59996,M,Do they understand that the bad attitudes they...,The worst?
59997,M,Dollars to donuts Target has a procedure for h...,"This was in 2010, so maybe."
59998,W,Do we deride the person who calls themself ath...,I do. And i just finished a


In [46]:
men_data['q_pronoun'] = men_data.question.apply(pronouns)
men_data['q_has_or'] = men_data.question.apply(has_or)
men_data['q_rhetoric'] = men_data.question.apply(rhetoric)
men_data['q_too_short'] = men_data.question.apply(too_short)

In [47]:
good_qs = men_data[~men_data.q_pronoun & ~men_data.q_has_or & ~men_data.q_rhetoric & ~men_data.q_too_short]
good_qs

,gender,question,response,q_pronoun,q_has_or,q_rhetoric,q_too_short
786,M,Do you mean a U lock?,Nope. There's no lockup.,False,False,False,False
857,W,Are you cool if I ask?,I'm okay :p,False,False,False,False
894,W,Are you a guy as well?,"If you're not, you're just a bro in disguise.",False,False,False,False
962,W,Do I live under a rock?,"No, i live on earth",False,False,False,False
965,W,Do you want my SSN too?,I've never had that happen in real life...,False,False,False,False
...,...,...,...,...,...,...,...
59869,M,Do you think that septum shaming in that scena...,They can't shame him because he is the only me...,False,False,False,False
59898,M,Didnt a recent study conclude that the demogra...,This is pretty accurate. They showed that the ...,False,False,False,False
59919,M,Was that grown man being paid to broadcast his...,That would be pretty unethical to tell other p...,False,False,False,False
59963,W,Do you want to put your son through the possib...,"Oh god, it just seems like that",False,False,False,False


In [48]:
len(good_qs.question.unique()) / len(men_data.question.unique())

0.24196306429548564

In [49]:
len(good_qs.question.unique())

1415

In [50]:
good_qs.gender.value_counts()

W    7210
M    6960
Name: gender, dtype: int64

In [51]:
m_good_qs = good_qs[good_qs.gender == 'M'].sample(500)
w_good_qs = good_qs[good_qs.gender == 'W'].sample(500)

In [52]:
m_good_qs[['gender', 'question', 'response']].to_csv('men_responses.tsv', sep='\t', index=False)
w_good_qs[['gender', 'question', 'response']].to_csv('women_responses.tsv', sep='\t', index=False)